In [1]:
import numpy as np

In [2]:
X_Test = np.load('data/test_features.npy')
Y_Test = np.load('data/test_labels.npy')

In [4]:
Y_Test.shape

(2000,)

In [4]:
X_Valid = np.load('data/validation_features.npy')
Y_Valid = np.load('data/validation_labels.npy')

In [5]:
X_Train = np.stack((np.load('data/training_features1.npy'), np.load('data/training_features2.npy')), axis=0)
Y_Train = np.stack((np.load('data/training_labels1.npy'), np.load('data/training_labels2.npy')), axis=0)

In [6]:
X_Train = X_Train.reshape((16000,209429))

In [7]:
X_Train[0].shape

(209429,)

In [8]:
Y_Train = Y_Train.reshape((16000))

In [9]:
test_index = np.load('index/test_index.npy')

In [12]:
test_index

array([ 3220, 17774, 15275, ..., 12195, 12244,  7390])

In [3]:
map_file = open("word_index_mapping.txt","r") 

In [4]:
word_index_map = {}
content = map_file.readlines()

In [5]:
for line in content:
    idx1 = line.find(',')
    idx2 = line.find('\n')
    word = line[:idx1]
    index = line[idx1+2:idx2-1]
    word_index_map[word] = int(index)

In [6]:
index_word_map = {}
for line in content:
    idx1 = line.find(',')
    idx2 = line.find('\n')
    word = line[:idx1]
    index = line[idx1+2:idx2-1]
    index_word_map[int(index)] = word

In [7]:
word_index_map['NOODLES']

16821

In [14]:
index_word_map[38]

'OF'

In [9]:
X_Test.shape

(2000, 209429)

In [10]:
import keras

Using TensorFlow backend.


In [11]:
model = keras.models.load_model('models1.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [14]:
Y_Pred = model.predict(X_Test, batch_size=4000)
Y_Pred = Y_Pred>0.5
#Y_Test = Y_Test.reshape((2000,1))
#test_acc = (1.0*np.sum(Y_Pred==Y_Test))/(1.0*Y_Test.shape[0])
#print('Test accuracy = '+str(test_acc)+' on '+str(Y_Test.shape[0])+' examples.')

In [15]:
Y_Test = Y_Test.reshape((2000,1))
test_acc = (1.0*np.sum(Y_Pred==Y_Test))/(1.0*Y_Test.shape[0])
print('Test accuracy = '+str(test_acc)+' on '+str(Y_Test.shape[0])+' examples.')

Test accuracy = 0.97 on 2000 examples.


In [16]:
np.sum(np.logical_and(Y_Pred, Y_Test))

938

In [17]:
Y_Test

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [18]:
false_positive = np.logical_and(Y_Pred==True, Y_Test==False)
false_negative = np.logical_and(Y_Pred==False, Y_Test==True)

In [19]:
np.sum(false_positive)
#np.sum(false_negative)

15

In [20]:
np.nonzero(false_positive)[0]

array([  24,   97,  184,  370,  408,  964, 1015, 1058, 1068, 1102, 1255,
       1357, 1678, 1901, 1955])

In [21]:
np.nonzero(false_negative)[0]

array([ 108,  140,  151,  171,  212,  224,  238,  260,  281,  358,  391,
        470,  599,  661,  705,  758,  801,  822,  859, 1020, 1031, 1235,
       1272, 1303, 1317, 1338, 1343, 1348, 1366, 1390, 1395, 1455, 1535,
       1543, 1595, 1604, 1695, 1722, 1736, 1794, 1802, 1858, 1891, 1938,
       1968])

In [80]:
real = X_Test[np.nonzero(Y_Test)[0]]
fake = X_Test[np.nonzero(Y_Test==False)[0]]

In [63]:
def getFrequentWords(document, k, index_word_map):
    ind = np.argpartition(document, -k)[-k:]
    max_index = np.argmax(document)
    words = []
    for index in ind:
        words.append(index_word_map[index])
    return words

In [89]:
document = fake[0]
getFrequentWords(document, 5, index_word_map)

['IS', 'TO', 'TRUMP', 'AND', 'BLACK']

In [97]:
document = real[8]
getFrequentWords(document, 5, index_word_map)

['AND', 'TO', 'THE', 'IN', 'OF']

In [104]:
def createDocument(content, word_index_map):
    document = [0 for i in range(209429)]
    words = content.split(' ')
    for i in range(len(words)):
        words[i] = words[i].upper()
        word = words[i]
        if word[-1] in '!,.:;?()_=-+@#$%^&*\'':
            words[i] = word[:-1]
        if words[i] in word_index_map:
            document[word_index_map[words[i]]] += 1
    return np.array(document).reshape((1,209429))

In [170]:
content = '''William Barr, Democrats Clash Over Robert Mueller’s Report By Sadie Gurman, Byron Tau and Kristina Peterson
Updated May 1, 2019 7:50 p.m. ET

WASHINGTON—Attorney General William Barr criticized Robert Mueller’s decision not to reach a conclusion about whether President Trump obstructed justice during a contentious hearing that laid bare a rift between him and the special counsel over the politically charged investigation.
In his first congressional testimony since releasing a redacted version of Mr. Mueller’s 448-page report, Mr. Barr faced pointed criticism from Senate Democrats over his handling of the findings on Russian election interference in 2016.
“If [Mr. Mueller] felt he shouldn’t go down a path of making a traditional prosecutive decision, then he shouldn’t have investigated,” Mr. Barr told the Senate Judiciary Committee Wednesday. “That was the time to pull up.”
Meanwhile, the Justice Department late Wednesday told House Democrats that Mr. Barr wouldn’t appear at a Judiciary Committee hearing scheduled for Thursday over disagreements about the format of the appearance—and that an unredacted version of the Mueller report, which had been subpoenaed by the committee, wouldn’t be provided.
SHARE YOUR THOUGHTS
What do you think of Attorney General William Barr’s handling of the Mueller report? Join the conversation below.
Rep. Jerrold Nadler (D., N.Y.), who chairs that committee, also threatened to hold the attorney general in contempt of Congress—a prelude to a court battle—for his continued refusal to turn over the unredacted Mueller report, a potentially dramatic escalation of tensions between congressional Democrats and the Trump administration.
Wednesday’s Senate hearing offered a dramatic public display of the behind-the-scenes jockeying to give Mr. Mueller’s findings their proper airing. It came just after the Justice Department released a March 27 letter from Mr. Mueller to the attorney general, saying Mr. Barr’s early public portrayals of the report’s main conclusions failed to fully capture the work of the special counsel’s office.
At the hearing, Mr. Barr called Mr. Mueller’s letter “a bit snitty” and suggested it was “written by one of his staff people.” Mr. Barr said he called Mr. Mueller the day after receiving it.
“I said, ‘Bob, what’s with the letter? Why don’t you just pick up the phone and call me if there’s an issue?’ ” Mr. Barr said.
Mr. Mueller “said his concern focused on his explanation of why he did not reach a conclusion on obstruction and he wanted more put out on that issue,” said Mr. Barr, recounting his conversation with Mr. Mueller. “He was very clear that he was not suggesting we had misrepresented his report.”
Democrats didn’t agree on that characterization. Sen. Richard Blumenthal (D., Conn.) called the letter “an extraordinary act, a career prosecutor rebuking the attorney general of the United States.”
Mr. Mueller’s letter was sent after the attorney general released his analysis of the investigation’s main conclusions in a four-page memo to Congress on March 24. Mr. Barr said he invited the special counsel to review the memo but that Mr. Mueller declined.
When Mr. Mueller, driven largely by Justice Department policy against indicting a sitting president, declined in his report to make a recommendation about whether Mr. Trump obstructed justice during the investigation, Mr. Barr determined the special counsel’s evidence was insufficient to establish Mr. Trump committed a crime.
Mr. Barr said Mr. Mueller’s reasoning on not pursuing an obstruction charge wasn’t immediately clear to him when the special counsel first told officials on March 5 that he wouldn’t be reaching a conclusion. The report detailed 10 episodes of potential obstruction of justice and made clear Mr. Mueller wasn’t exonerating the president.
“We didn’t really get a clear understanding of the reasoning,” Mr. Barr said, adding that the 448-page report offered no concise explanation, either. “That’s one of the reasons why I didn’t want to put words in Bob Mueller’s mouth” before releasing the entire report.
In the tense hearing that also highlighted partisan divisions on the Senate Judiciary Committee, Democrats pressed Mr. Barr on why he wasn’t more forthcoming about his disagreements with Mr. Mueller. Republicans on the panel repeatedly asked for information about whether the Federal Bureau of Investigation appropriately handled the early stages of the investigation, echoing the GOP president’s longstanding criticism of the Justice Department’s probe. 
Some Republicans asked about the status of a review Mr. Barr previously disclosed into what he termed “spying” on people affiliated with the Trump campaign. Among other allegations, the review is examining whether the FBI had appropriately obtained surveillance warrants against a former Trump foreign-policy adviser.
“To the extent there was any overreach, I believe it was a few people in the upper echelons of the bureau and perhaps the department, but those people are no longer there,” Mr. Barr told Sen. Mike Lee (R., Utah).

Multiple Democrats questioned whether Mr. Barr misled lawmakers at a House hearing last month, when he denied knowing of concerns by Mr. Mueller’s team about the attorney general’s four-page summary two days after receiving the report itself from the special counsel. At the time of that hearing, Mr. Barr had received two letters from Mr. Mueller and had had his phone conversation with him, all in connection with whether Mr. Barr had misrepresented the report. The Fight Over the Mueller Report Moves to Congress

Congress is taking steps to see the unredacted Mueller report and the evidence that supports it. WSJ’s Siobhan Hughes looks at the political options that could play out in Congress. Photo Illustration: Adele Morgan
“Why did you say you were not aware of concerns when weeks before your testimony Mr. Mueller had expressed concerns to you?” Sen. Patrick Leahy (D., Vt.) asked. Mr. Barr said he was accurately responding that he didn’t know what concerns he was being asked about.
Mr. Mueller in his March 27 made it clear he preferred the report’s introduction and executive summaries—which his team had redacted so the material could be viewed by the public—be quickly released.
But Mr. Barr testified Wednesday that, while he asked Mr. Mueller to make redactions in order to hasten the report’s release, he found when he received it on March 22 that the summaries still contained material that couldn’t be made public. He said he knew it would take weeks to make the edits.
“I told Bob I wasn’t interested in putting out summaries,” he said. “I wanted to put out the whole report.”
The hearing at times turned fiery. Asked by Sen. Kamala Harris (D., Calif.) if he would consult with career Justice Department officials about whether he would recuse himself from the more than a dozen investigations referred to other jurisdictions by the Mueller probe, Mr. Barr said he had no conflict of interest that would necessitate his recusal.

“I think the American public has seen quite well that you are biased in this situation and you have not been objective,” said Ms. Harris, who is seeking the 2020 Democratic presidential nomination.
In another exchange, after Sen. Mazie Hirono (D., Hawaii) explained why she voted against confirming Mr. Barr as attorney general, she said: “I expected you would try to protect the president and indeed you did,” before calling on Mr. Barr to resign.
That drew a slap from Sen. Lindsey Graham of South Carolina, the panel’s GOP chairman.
“Seven minutes and you’ve slandered this man from top to bottom,” Mr. Graham said.
Earlier, Mr. Graham said the president couldn’t have obstructed justice if there was no underlying crime committed by Mr. Trump’s campaign associates. While the Mueller team found repeated contacts between the Trump campaign and Moscow-linked entities and that the Trump team expected it would benefit “from information stolen and released through Russian efforts,” the probe didn’t establish that members of the Trump campaign conspired with the Russian government.
“Attempted obstruction of justice of a crime that never occurred seems to be the new standard around here, to me it doesn’t make any sense,” Mr. Graham said.
Under the law, obstruction doesn’t require a successful effort. Nor does a prosecutor need to prove there was an underlying crime that a suspect was covering up.
The House Judiciary panel in Thursday’s hearing wanted to have staff attorneys handle some of the questioning of the attorney general, but Mr. Barr balked.
“The committee has the right to determine its own procedures,” Mr. Nadler said after getting the news that Mr. Barr wouldn’t appear. “The Congress cannot permit the executive branch, we cannot permit the administration, to dictate to Congress how we operate.”
A Justice Department spokeswoman said that “Chairman Nadler’s insistence on having staff question the Attorney General, a Senate-confirmed Cabinet member, is inappropriate.”
Write to Sadie Gurman at sadie.gurman@wsj.com, Byron Tau at byron.tau@wsj.com and Kristina Peterson at kristina.peterson@wsj.com
Appeared in the May 2, 2019, print edition as 'Attorney General, Democrats Clash Over Mueller Report. 
'''
document = createDocument(content, word_index_map)
model.predict(document, batch_size=4000)

array([[0.51925415]], dtype=float32)

In [146]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [55]:
def load_dataset():
    X_Test = np.load('data/test_features.npy')
    Y_Test = np.load('data/test_labels.npy')
    X_Valid = np.load('data/validation_features.npy')
    Y_Valid = np.load('data/validation_labels.npy')
    X_Train = np.stack((np.load('data/training_features1.npy'), np.load('data/training_features2.npy')), axis=0)
    Y_Train = np.stack((np.load('data/training_labels1.npy'), np.load('data/training_labels2.npy')), axis=0)
    Y_Test = Y_Test.reshape((2000,1))
    Y_Valid = Y_Valid.reshape((2000,1))
    X_Train = X_Train.reshape((16000,209429))
    Y_Train = Y_Train.reshape((16000,1))
    return X_Train, Y_Train, X_Valid, Y_Valid, X_Test, Y_Test

In [179]:
def model(input_size=(209429,)):
    inputs = Input(input_size)
    X1 = Dense(128, activation='relu')(inputs)
    X2 = Dense(128, activation='relu')(X1)
    X3 = Dense(64, activation='relu')(X2)
    X4 = Dense(64, activation='relu')(X3)
    X5 = Dense(32, activation='relu')(X4)
    X6 = Dense(32, activation='relu')(X5)
    X7 = Dense(16, activation='relu')(X6)
    X8= Dense(16, activation='relu')(X7)
    X9 = Dense(4, activation='relu')(X8)
    X10 = Dense(4, activation='relu')(X9)
    X11 = Dense(1, activation='relu')(X10)
    outputs = Dense(1, activation='sigmoid')(X11)
    
    model = Model(input=inputs, output=outputs)
    model.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'], )
    model.summary()

    return model

In [180]:
new_model = model()
new_model.fit(X_Train, Y_Train, epochs=10, batch_size=300, validation_data=(X_Valid,Y_Valid))

/nas/longleaf/home/mingzhi/.local/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 209429)            0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               26807040  
_________________________________________________________________
dense_31 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_32 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_33 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_34 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_35 (Dense)             (None, 32)                1056      
__________

KeyboardInterrupt: 

In [60]:
model.save('models1.h5')

In [178]:
Y_Pred = new_model.predict(X_Test, batch_size=4000)
Y_Pred = Y_Pred>0.5
Y_Test = Y_Test.reshape((2000,1))
test_acc = (1.0*np.sum(Y_Pred==Y_Test))/(1.0*Y_Test.shape[0])
print('Test accuracy = '+str(test_acc)+' on '+str(Y_Test.shape[0])+' examples.')

Test accuracy = 0.9605 on 2000 examples.


In [51]:
KK = Y_Pred == Y_Test

In [46]:
np.sum(Y_Test)

983

In [52]:
KK.shape

(2000, 2000)

In [53]:
Y_Pred.shape

(2000, 1)